## Computer Vision
### Read, Write and Display a video using OpenCV

In [1]:
import numpy as np
import cv2 as cv 
import os

In [2]:
# change the path to match on your machine
base_folder = 'easy'
path_video1 = os.path.join(base_folder, "1.mp4")

### Reading and displaying a video

In [3]:
# Open the video 
cap = cv.VideoCapture(path_video1) 

if cap.isOpened() == False: 
    print("Error opening video stream or file") 
    
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
print('frame_width = ' + str(frame_width))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
print('frame_height = ' + str(frame_height))

length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
print("total number of frames = " + str(length))

fps = int(cap.get(cv.CAP_PROP_FPS))
print("frames per second = " + str(fps))

frame_width = 1280
frame_height = 720
total number of frames = 256
frames per second = 30


In [4]:
# Display the video
current_frame = 0
max_number_of_frame_to_run = 750

while cap.isOpened(): 
    
    ret, frame = cap.read() # Read the frame
    if ret is True:
        current_frame = current_frame + 1 
        cv.imshow("Frame", frame)
        
        if current_frame > max_number_of_frame_to_run:
            break
            
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# after playing the video, release the video capture    
cap.release()
# close all the frames
cv.destroyAllWindows()

In [4]:
# This function takes the video path and returns the a list of frames.
def read_frames(video_path):
    frames = []
    cap = cv.VideoCapture(video_path)  
    if cap.isOpened() == False: 
        print("Error opening video stream or file") 
        return frames
    
    while cap.isOpened():  
        ret, frame = cap.read() # Read the frame
        if ret is True:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

### Writing a video

In [5]:
# We are going to write the same video at 1 fps, first we need to read the frames.
frames = read_frames(path_video1)

# here we have the extensions and the fourcc for each of it
video_extension_and_fourcc_dict = {'avi': cv.VideoWriter_fourcc('M', 'J', 'P', 'G'),
                                   'mp4': 0x7634706d}   
 
# We need to create a VideoWriter object. 
# First, we should specify the output file name with its format (eg: 1_fps_1.mp4). 
# we should specify the FourCC code and the number of frames per second (FPS). 
# Lastly, the frame size should be passed (width, height).

video_output_name = "1_fps_1.mp4"
output_video = cv.VideoWriter(video_output_name, video_extension_and_fourcc_dict["mp4"], 1,
                              (frames[0].shape[1], frames[0].shape[0]))

num_frames = len(frames)
# We know that the first video has 30 fps.
for i in range(0, num_frames, 30):
    output_video.write(frames[i]) # writing the frame

# don't forget to release the video writer
output_video.release()

## Project 2 - Snooker
We are going to detect the snooker table and the snooker balls based on the color using the hSV color space.

In [6]:
# get a feeling about detecting the snooker table and the snooker balls based on the color using the hSV color space
# conversion from BGR to HSV
# https://docs.opencv.org/trunk/df/d9d/tutorial_py_colorspaces.html

# the snooker table is green
bgr_green = np.uint8([[[0, 255, 0]]])
hsv_green = cv.cvtColor(bgr_green, cv.COLOR_BGR2HSV)
print("hsv_green", hsv_green )

# the cue ball is white
bgr_white = np.uint8([[[255, 255, 255]]])
hsv_white = cv.cvtColor(bgr_white, cv.COLOR_BGR2HSV)
print("hsv_white", hsv_white)

# the other snooker balls are red, black, pink, blue, brown, green (dark), yellow
bgr_red = np.uint8([[[0, 0, 255]]])
hsv_red = cv.cvtColor(bgr_red, cv.COLOR_BGR2HSV)
print("hsv_red", hsv_red )

bgr_blue = np.uint8([[[255 ,0 ,0]]])
hsv_blue = cv.cvtColor(bgr_blue, cv.COLOR_BGR2HSV)
print("hsv_blue", hsv_blue )

hsv_green [[[ 60 255 255]]]
hsv_white [[[  0   0 255]]]
hsv_red [[[  0 255 255]]]
hsv_blue [[[120 255 255]]]


In [17]:
def find_color_values_using_trackbar(frame):
    # convert the frame from BGR to HSV
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # we need this function as a callback 
    def nothing(x):
        pass

    cv.namedWindow("Trackbar") 
    cv.createTrackbar("LH", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LS", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LV", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("UH", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("US", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("UV", "Trackbar", 255, 255, nothing)
    
    
    while True:

        l_h = cv.getTrackbarPos("LH", "Trackbar")
        l_s = cv.getTrackbarPos("LS", "Trackbar")
        l_v = cv.getTrackbarPos("LV", "Trackbar")
        u_h = cv.getTrackbarPos("UH", "Trackbar")
        u_s = cv.getTrackbarPos("US", "Trackbar")
        u_v = cv.getTrackbarPos("UV", "Trackbar")


        l = np.array([l_h, l_s, l_v])
        u = np.array([u_h, u_s, u_v])
        mask_table_hsv = cv.inRange(frame_hsv, l, u)        

        res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)    
        cv.imshow("Frame", frame)
        cv.imshow("Mask", mask_table_hsv)
        cv.imshow("Res", res)

        if cv.waitKey(25) & 0xFF == ord('q'):
                break


    cv.destroyAllWindows()
    

In [18]:
# use a trackbar to get right values for the table

path_video = os.path.join(base_folder, "1.mp4")
cap = cv.VideoCapture(path_video)
cap.set(1, 15)  # where frame_no is the frame you want
ret, frame = cap.read()  # Read the frame
cap.release()  # close the VideoCapture
find_color_values_using_trackbar(frame)


KeyboardInterrupt: 

In [23]:
# use a trackbar to get right values for balls 
# extract the table
# these are the right colors for green
def find_table(frame, show_results=True):

    low_green = (46, 100, 0)
    high_green = (85, 255, 255)

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # get the mask for green
    mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)
    if show_results:
        cv.imshow('mask_table_hsv', mask_table_hsv)
        cv.waitKey(0)
        

    kernel = np.ones((15, 15), np.uint8)
    mask_table_hsv = cv.dilate(mask_table_hsv, kernel, iterations=2)
    if show_results: 
        cv.imshow('mask_table_hsv_dilatation', mask_table_hsv)
        cv.waitKey(0)

    mask_table_hsv = cv.erode(mask_table_hsv, kernel, iterations=2)
    if show_results: 
        cv.imshow('mask_table_hsv_erosion', mask_table_hsv)
        cv.waitKey(0)

    table = cv.bitwise_and(frame, frame, mask=mask_table_hsv)  
    
    if show_results: 
        cv.imshow('table', table) 
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    return table, mask_table_hsv

In [24]:
table, mask_table_hsv = find_table(frame)

In [25]:
find_color_values_using_trackbar(table)

KeyboardInterrupt: 

In [10]:
def detect_balls_using_hsv(table, show_results=True):
    low_white = np.array([80, 140, 65])
    high_white = np.array([255, 255, 255])        
    mask_white = cv.inRange(table, low_white, high_white)
    white_ball = cv.bitwise_and(table,table, mask=mask_white)    

    low_red = np.array([0, 0, 26])
    high_red = np.array([26, 42, 255])
    mask_red = cv.inRange(table,low_red,high_red)
    red_ball = cv.bitwise_and(table,table,mask=mask_red) 
 
    low_blue = np.array([91,0,0])
    high_blue = np.array([180,135,55])
    mask_blue = cv.inRange(table, low_blue, high_blue)
    blue_ball = cv.bitwise_and(table, table, mask=mask_blue)
 
    low_pink = np.array([49, 0, 187])
    high_pink = np.array([255, 162, 255])        
    mask_pink = cv.inRange(table, low_pink, high_pink)
    pink_ball = cv.bitwise_and(table, table,mask=mask_pink) 

    low_dark_green = np.array([37, 23, 0])
    high_dark_green = np.array([84, 255, 30])        
    mask_dark_green = cv.inRange(table, low_dark_green, high_dark_green)

    low_brown = np.array([0, 52, 37])
    high_brown = np.array([45, 105, 166])        
    mask_brown = cv.inRange(table, low_brown, high_brown)

    low_yellow = np.array([0, 107, 68])
    high_yellow = np.array([114, 255, 255])        
    mask_yellow = cv.inRange(table, low_yellow, high_yellow) 

    low_black = np.array([0, 0, 0])
    high_black = np.array([45, 38, 65])        
    mask_black = cv.inRange(table, low_black, high_black)

    
    mask_balls = mask_white + mask_red + mask_blue + mask_pink + mask_dark_green + mask_brown + mask_yellow + mask_black

    balls = cv.bitwise_and(table,table,mask=mask_balls) 
    if show_results:
        cv.imshow("Table", table)
        cv.imshow("BALLS", balls)
        cv.imshow("Mask balls", mask_balls)

        cv.waitKey(0)
        cv.destroyAllWindows()
        
    return table, balls, mask_balls
    

In [11]:
_ = detect_balls_using_hsv(table, True)

In [14]:
# put all the things together until now and see if it's working on the video
path_video = os.path.join(base_folder, "4.mp4")
cap = cv.VideoCapture(path_video)

current_frame = 0
max_number_of_frame_to_run = 750
show_results = False 

while cap.isOpened():
    # Read the frame
    ret, frame = cap.read() 
    if ret == True:        
        current_frame = current_frame + 1
        cv.imshow("Frame", frame)
        table, mask_table_hsv = find_table(frame, show_results)      
        contours, _ = cv.findContours(mask_table_hsv, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)   
        max_area = 0        
        img_contour = np.zeros(table.shape, np.uint8)
        
        # take the largest in area contour
        if len(contours) > 0:
            cnt = contours[0]
            for i, cont in enumerate(contours):   
                contour_area = cv.contourArea(cont)            
                if contour_area > max_area:
                    max_area = contour_area
                    cnt = contours[i]     
                    
            epsilon = 0.01 * cv.arcLength(cnt, True) # contour perimeter
            approx = cv.approxPolyDP(cnt, epsilon, True) # get the approximation
            hull = cv.convexHull(cnt)
            cv.drawContours(img_contour, cnt, -1, (0, 255, 0), 3)
            cv.drawContours(img_contour, [approx], -1, (0, 0, 255), 3)
            cv.drawContours(img_contour, hull, -1, (255, 0, 0), 3) # only displays a few points as well.
            if show_results:
                cv.imshow('img_contour', img_contour) 
            
        table, balls, mask_balls = detect_balls_using_hsv(table, show_results)    
        cv.imshow('balls', balls)  
                  
        if current_frame > max_number_of_frame_to_run:
            break
            
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
     
    else:
        break
    
# after playing the video, release the video capture    
cap.release()
# close all the frames
cv.destroyAllWindows()
 

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
cv.destroyAllWindows()